<a href="https://colab.research.google.com/github/iresh-rajitha/Character-seperate-using-CV/blob/main/Selenium_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install selenium-wire

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pac

In [2]:
# prompt: get a file from google drive

from google.colab import drive
drive.mount('/content/drive')

!cp /content/drive/MyDrive/sinhala_fonts/font_names.txt .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# get downloaded_fonts.txt file from google drive if not exist
import os
if not os.path.exists("downloaded_fonts.txt"):
  !cp /content/drive/MyDrive/sinhala_fonts/downloaded_fonts.txt .

In [4]:
# Create the downloads folder if it doesn't exist
if not os.path.exists("downloads"):
    os.makedirs("downloads")

In [5]:
# Step 3: Set up the Selenium WebDriver with Chrome options
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import traceback
import time

# Set up Chrome options
chrome_options = Options()
# chrome_options.add_argument("--incognito")  # Open Chrome in incognito mode
chrome_options.add_argument("--headless")  # Ensure the browser runs in headless mode
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
# chrome_options.binary_location = "/usr/bin/chromium-browser"

# Set the path to your WebDriver executable
driver_path = "/usr/bin/chromedriver"
download_link = ""
# Initialize the WebDriver
# service = Service(driver_path)
def get_font_link(font):
  driver = webdriver.Chrome(options=chrome_options)
  try:
      # Open a website
      driver.get(f'https://sinhala-fonts.org/fonts/{font}')

      # Wait until the page is loaded
      WebDriverWait(driver, 30).until(
          EC.presence_of_element_located((By.TAG_NAME, "body"))
      )

      # Execute JavaScript to trigger the download
      java_script = """
          document.getElementById("id_method").value = "app";
          document.getElementById("fontdownloadsubmit").click();
          """
      driver.execute_script(java_script)

      # Wait for the href of the download link to change
      WebDriverWait(driver, 30).until(
          EC.text_to_be_present_in_element_attribute(
              (By.XPATH, "//a[starts-with(@href, '/downloadfiles/')]"),
              "href",
              "/downloadfiles/"
          )
      )

      # Get all a tags href
      a_tags = driver.find_elements(By.XPATH, "//a")
      for a_tag in a_tags:
          # Filter link start with https://sinhala-fonts.org/downloadfiles/
          if a_tag.get_attribute("href").startswith("https://sinhala-fonts.org/downloadfiles/"):
              print(a_tag.get_attribute("href"))
              download_link = a_tag.get_attribute("href")


  except Exception as e:
      print(f"Error while using webdriver: {e}")
      traceback.print_exc()
  finally:
      driver.delete_all_cookies()  # Clears all cookies
      driver.execute_script("window.localStorage.clear();")  # Clears local storage
      driver.execute_script("window.sessionStorage.clear();")  # Clears session storage
      driver.quit()
  return download_link

In [8]:
# prompt: read the each line of the font_names.txt file and make a font name array and remove the names which in downloaded_fonts.txt

def update_font_names():
    with open('font_names.txt', 'r') as f:
        font_names = [line.strip() for line in f]

    with open('downloaded_fonts.txt', 'r') as f:
        downloaded_fonts = [line.strip() for line in f]

    # Remove downloaded fonts from the font_names list
    font_names = [font for font in font_names if font not in downloaded_fonts]
    return font_names

In [ ]:
from tqdm.notebook import tqdm
import requests
# Initialize a counter for downloaded fonts


# Create an empty list to store downloaded font names
downloaded_fonts = []
# Loop through the font names
def download_fonts():
  font_names = update_font_names()
  downloaded_count = 0
  download_link = ""
  for font in tqdm(font_names, desc='Processing fonts'):
      try:
          # Call the function to get the download link
          download_link = get_font_link(font)
          response = requests.get(download_link)

          if response.headers.get('Content-Type') == 'application/zip':
            with open(f"downloads/{font}.zip", "wb") as f:
              f.write(response.content)
          else:
            raise Exception("Not a zip file")

          # write to downloaded_fonts.txt as append new line
          with open("downloaded_fonts.txt", "a") as file:
              file.write(font + "\n")

          # Add the font name to the list of downloaded fonts
          downloaded_fonts.append(font)

          # Increment the downloaded count
          downloaded_count += 1

          # Print the progress
          print(f"Downloaded {downloaded_count} fonts out of {len(font_names)}")

      except Exception as e:
          print(f"Error downloading font {font}: {e}")
          break

      finally:
          !cp -r /content/downloads /content/drive/MyDrive/sinhala_fonts
          with open('/content/drive/MyDrive/sinhala_fonts/downloaded_fonts.txt', 'w') as f:
              for font in downloaded_fonts:
                  f.write(font + '\n')

# call download_fonts when fail and call it back in 50 secnds
for i in range(100):
  print(f'attempts {i}')
  try:
    download_fonts()
  except Exception as e:
    print(f"Error downloading fonts: {e}")
    time.sleep(50)
    download_fonts()

attempts 0


Processing fonts:   0%|          | 0/962 [00:00<?, ?it/s]

Error while using webdriver: Message: 
Stacktrace:
#0 0x5b5c3dc7a8da <unknown>
#1 0x5b5c3d948e50 <unknown>
#2 0x5b5c3d998644 <unknown>
#3 0x5b5c3d998931 <unknown>
#4 0x5b5c3d9de534 <unknown>
#5 0x5b5c3d9bd4bd <unknown>
#6 0x5b5c3d9db9c6 <unknown>
#7 0x5b5c3d9bd233 <unknown>
#8 0x5b5c3d98b093 <unknown>
#9 0x5b5c3d98c09e <unknown>
#10 0x5b5c3dc41b3b <unknown>
#11 0x5b5c3dc45af1 <unknown>
#12 0x5b5c3dc2d705 <unknown>
#13 0x5b5c3dc46662 <unknown>
#14 0x5b5c3dc128df <unknown>
#15 0x5b5c3dc696d8 <unknown>
#16 0x5b5c3dc698a2 <unknown>
#17 0x5b5c3dc796cc <unknown>
#18 0x7e6281e05ac3 <unknown>



Traceback (most recent call last):
  File "<ipython-input-5-a8ecd85a1293>", line 43, in get_font_link
    WebDriverWait(driver, 30).until(
  File "/usr/local/lib/python3.10/dist-packages/selenium/webdriver/support/wait.py", line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
#0 0x5b5c3dc7a8da <unknown>
#1 0x5b5c3d948e50 <unknown>
#2 0x5b5c3d998644 <unknown>
#3 0x5b5c3d998931 <unknown>
#4 0x5b5c3d9de534 <unknown>
#5 0x5b5c3d9bd4bd <unknown>
#6 0x5b5c3d9db9c6 <unknown>
#7 0x5b5c3d9bd233 <unknown>
#8 0x5b5c3d98b093 <unknown>
#9 0x5b5c3d98c09e <unknown>
#10 0x5b5c3dc41b3b <unknown>
#11 0x5b5c3dc45af1 <unknown>
#12 0x5b5c3dc2d705 <unknown>
#13 0x5b5c3dc46662 <unknown>
#14 0x5b5c3dc128df <unknown>
#15 0x5b5c3dc696d8 <unknown>
#16 0x5b5c3dc698a2 <unknown>
#17 0x5b5c3dc796cc <unknown>
#18 0x7e6281e05ac3 <unknown>



Error downloading font 4u-araliya-plain: local variable 'download_link' referenced before assignment
attempts 1


...

https://sinhala-fonts.org/downloadfiles/zippedfontstyles/4/u/4u-chami-plain.zip?st=WrPh2_AYMu-fatSJVhCH4Q&e=1724972700
Downloaded 1 fonts out of 954
Error while using webdriver: Message: 
Stacktrace:
#0 0x579edfad68da <unknown>
#1 0x579edf7a4e50 <unknown>
#2 0x579edf7f4644 <unknown>
#3 0x579edf7f4931 <unknown>
#4 0x579edf83a534 <unknown>
#5 0x579edf8194bd <unknown>
#6 0x579edf8379c6 <unknown>
#7 0x579edf819233 <unknown>
#8 0x579edf7e7093 <unknown>
#9 0x579edf7e809e <unknown>
#10 0x579edfa9db3b <unknown>
#11 0x579edfaa1af1 <unknown>
#12 0x579edfa89705 <unknown>
#13 0x579edfaa2662 <unknown>
#14 0x579edfa6e8df <unknown>
#15 0x579edfac56d8 <unknown>
#16 0x579edfac58a2 <unknown>
#17 0x579edfad56cc <unknown>
#18 0x7f9d9a803ac3 <unknown>



Traceback (most recent call last):
  File "<ipython-input-5-a8ecd85a1293>", line 43, in get_font_link
    WebDriverWait(driver, 30).until(
  File "/usr/local/lib/python3.10/dist-packages/selenium/webdriver/support/wait.py", line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
#0 0x579edfad68da <unknown>
#1 0x579edf7a4e50 <unknown>
#2 0x579edf7f4644 <unknown>
#3 0x579edf7f4931 <unknown>
#4 0x579edf83a534 <unknown>
#5 0x579edf8194bd <unknown>
#6 0x579edf8379c6 <unknown>
#7 0x579edf819233 <unknown>
#8 0x579edf7e7093 <unknown>
#9 0x579edf7e809e <unknown>
#10 0x579edfa9db3b <unknown>
#11 0x579edfaa1af1 <unknown>
#12 0x579edfa89705 <unknown>
#13 0x579edfaa2662 <unknown>
#14 0x579edfa6e8df <unknown>
#15 0x579edfac56d8 <unknown>
#16 0x579edfac58a2 <unknown>
#17 0x579edfad56cc <unknown>
#18 0x7f9d9a803ac3 <unknown>



Error downloading font 4u-derana-x: local variable 'download_link' referenced before assignment
attempts 11


Processing fonts:   0%|          | 0/953 [00:00<?, ?it/s]

https://sinhala-fonts.org/downloadfiles/zippedfontstyles/4/u/4u-derana-x.zip?st=FWe28Ysm2J4WYBKgz5IZyw&e=1724972759
